Cette base de donnée se définie comme "une application internet chargée de centraliser l'ensemble des données sur les incendies de forêt sur le territoire français depuis 2006 et de mettre l'ensemble de cette information à disposition du public et des services de l'Etat."  
  
Elle est éditée par l'IGN (Institut National de l'Information Géographique et Forestières).

In [ ]:
import pandas as pd 
import numpy as np 
from datetime import datetime

Dans cette première partie nous allons voir en détails ce que contiennent les bases de données que nous avons trouver lors de nos recherches.

## Projet Python pour la data science (2023-2024)
*Thomas Chen, Félix de Champs, David Premachandra*  


## Introduction : 
<div align="justify">
Comme le montre ce graphique du site Statista les surfaces de forêts qui partent chaque année en fumée n'ont cessé d'augmenter au cours de la dernière décenies.  
  
![graph_statista](/graph_statista.PNG)
  
Les incendies de forêts sont donc une préoccupation croissante des populations et des pouvoirs publiques. En effet, ces millieu d'hectares brûlés représentent non seulement une catastrophe écologique, quand on connaît l'importance des forêts pour la biodiversité ou pour la captation du $ CO_{2}$, mais aussi dans certains cas une catastrophe humaine et matériel puisque 80 % des incendies de forêts se déclenchent à moins de 50 m des habitations selon le Ministère de l'Ecologie.  
On comprend alors l'importance de comprendre les circonstances de ces incendies pour mieux les prévenir, comme par exemple essayer de déterminer précisément quelles conditions climatiques vont sont les plus propices à l'apparition d'un feu de forêts.  
Notre projet à donc pour objectif de donner la probabilité qu'un feu de forêt se déclenche dans le Sud-Est de la France, en fonction des conditions météo comme nous allons le voir par la suite.


## Présentation des bases de données principales


# 0/ Récupération des données 

- On commence par importer les modules pour le traitement des données : pandas pour manipuler les dataframes, numpy pour faire des opérations mathématiques et datetime pour la manipulation des dates.

- Ensuite on va charger les différentes bases de données sur python en les faisant passer du format csv au format dataframe de pandas:  
  
  - la base de données sur les incendies de forêt est disponible via ce [lien](https://bdiff.agriculture.gouv.fr/incendies).  
  - la base de données météo est disponible via ce [lien](https://meteonet.umr-cnrm.fr/dataset/data/SE/ground_stations/).
  - la base de données sur les communes est disponible via ce [lien](https://www.data.gouv.fr/fr/datasets/r/dbe8a621-a9c4-4bc3-9cae-be1699c5ff25).


In [ ]:
# Dataframe de la base de données sur les incendies: 
df_incendies = pd.read_csv(r"C:\Users\felix\Desktop\ensae-prog2A\data\Incendies.csv", sep = ';')

# Dataframe avec la base de données meteonet pour le quart sud-est :
df_meteo_SE = pd.read_csv(r'C:\Users\felix\Desktop\ensae-prog2A\data\meteonet\ground_stations\SE2018.csv')

# Dataframe avec juste les stations et leurs coordonnées GPS :
df_stations_SE = df_meteo_SE[['number_sta','lat','lon']]
df_stations_SE = df_stations_SE.drop_duplicates(subset='number_sta').reset_index()

# Dataframe avec la base de données permettant de faire la correspondance commune-coordonnées GPS :
df_communes = pd.read_csv(r"C:\Users\felix\Desktop\ensae-prog2A\data\communes.csv")
df_communes = df_communes[['code_commune_INSEE','latitude','longitude','nom_region']]
df_communes.columns = ['code_commune_INSEE', 'lat', 'lon', 'region']
# On se restrenint aux régions qui nous intéressent
liste_regions = ['Corse', "Provence-Alpes-Côte d'Azur", "Occitanie", "Auvergne-Rhône-Alpes", 'Nouvelle-Aquitaine']
df_communes = df_communes[df_communes['region'].isin(liste_regions)].drop_duplicates(subset = 'code_commune_INSEE').reset_index()
# Correction des codes communes INSEE des départements en 0X 
for i in df_communes.index :
    if len(df_communes['code_commune_INSEE'][i])==4 :
        df_communes['code_commune_INSEE'][i] = '0'+ df_communes['code_commune_INSEE'][i]


# 1/ Base de données BDIFF

les données sur les incendies en 2018 sont contenues dans le dataframe df_incendies qui possède 1928 lignes et 37 colonnes. Chaque ligne correspond à un incendie.

In [ ]:
print(df_incendies.shape)
print(df_incendies.columns)

On dispose de données relatives:
- au temps avec la date d'alerte 
- aux surfaces parcourues par le feu
- à l'origine de l'incendie 
- aux dégats et décès

### Regardons quelques statistiques descriptives sur les variables qualitatives relatives aux incendies:

#### On crée des colonnes mois et heure pour s'intéresser à l'occurence des incendies par rapport au temps.

In [ ]:
df_incendies['date'] = pd.to_datetime(df_incendies['Date de première alerte'])
from datetime import datetime

df_hist_incendies = df_incendies.copy()

df_hist_incendies['date']=pd.to_datetime(df_hist_incendies['date'])
df_hist_incendies['mois']=df_hist_incendies['date'].dt.month
mois_noms = { 1: 'janvier', 2: 'février', 3: 'mars', 4: 'avril', 5: 'mai', 6: 'juin', 
             7: 'juillet', 8: 'août', 9: 'septembre', 10: 'octobre', 11: 'novembre', 12: 'décembre'}

df_hist_incendies['mois'] = df_hist_incendies['mois'].replace(mois_noms) #remplacer les valeurs numériques des mois par leurs noms

df_hist_incendies['heure']=pd.to_datetime(df_hist_incendies['Date de première alerte']).dt.hour


#### Histogrammes du nombre d'incendies par mois et par heure

In [ ]:
"histogramme du nombre d'incendies par mois"
plt.figure(figsize=(15, 6))
df_hist_incendies['mois'].value_counts().plot(kind='bar', color='orange')
plt.title('Nombre d\'incendies par mois en 2018 dans le Sud-Est')
plt.xlabel('Mois')
plt.ylabel('Nombre d\'incendies')
plt.xticks(rotation=0)

"histogramme du nombre d'incendies par heure"
plt.figure(figsize=(15, 6))
df_hist_incendies['heure'].value_counts().sort_index().plot(kind='bar', color='orange')
plt.title('Nombre d\'incendies par heure en 2018 dans le Sud-Est')
plt.xlabel('heure')
plt.ylabel('Nombre d\'incendies')
plt.xticks(rotation=0)

plt.show()

<div align="justify">
En regardant l'occurence des incendies en 2018, on constate que les mois d'été connaissent bien plus d'incendies que les mois d'hiver et d'automne. Quant à l'heure, on remarque qu'un grand nombre d'incendies ont lieu dans l'après-midi. On peut supposer que cela est dû au climat d'été qui est d'avantage propice au déclenchement d'un feu de forêt. La variable heure qui correspond à l'heure du signalement de l'incendie doit vraisemblablement être proche de l'heure d'apparition du feu, on peut dire que les conditions climatiques dans l'après-midi sont plus favorables aux incendies que celles du matin ou de la nuit. De plus la concentration des heures d'incendies dans l'après-midi peut être liée à la fréquentation qui semble jouer un rôle assez important puisque la plupart des incendies ont une cause non naturelle.

In [ ]:
for col in ['Nature','Type de peuplement']: 
    df_incendies_18[col].value_counts().plot.pie(autopct='%1.1f%%')
    plt.title("répartition des incendies selon "+ col)
    plt.show()

En effet, 85% des incendies sont causés par l'homme. On observe également que certains types de peuplement c'est-à-dire une formation végétale spécifique sont plus touchés par les incendies. Le type de peuplement 1 qui représente 59% des incendies correspond aux Landes, garrigues et maquis.

# 2/ Base de données Meteonet

## Création de la base de données finale
Les données sont toutes rassemblées dans un dossier nommé "data".

- Les données du dataset de Météo France sont données toutes les 6 minutes pour chaque station sur l'ensemble de l'année. Nous avons donc décidé de les moyenner sur la journée afin réduire le nombre d'observations à une par jour et par station. Cela se justifie dans la mesure où les tendances intrajournalières des paramètres météorologiques n'ont pas beaucoup d'importance dans le cadre de notre modélisation.  
C'est donc le sens de ce dataframe `df_meteo_moy`.

In [ ]:
df_meteo_SE['date'] = pd.to_datetime(df_meteo_SE['date'])
df_meteo_moy = df_meteo_SE.groupby(['number_sta', pd.Grouper(key = 'date', freq = 'D')]).mean().reset_index()
display(df_meteo_moy.head())

- On va ensuite vouloir associer à chaque commune la station météorologique la plus proche pour pouvoir avoir accès à la météo journalière de la commune en question. Pour cela on a besoin de calculer la distance entre la position GPS de la commune et celle de la station météo, on utilise donc cette fonction : 

In [ ]:
def distance(lat1,lon1,lat2,lon2):
    """ 
    INPUT : les coordonnées GPS de deux points
    
    OUTPUT : la distance entre ces deux points à vol d'oiseau
    """
    return 6371*np.arccos(np.sin(lat1*np.pi/180)*np.sin(lat2*np.pi/180)+np.cos(lat1*np.pi/180)*np.cos(lat2*np.pi/180)*np.cos((lon2-lon1)*np.pi/180))


- La cellule suivante permet de d'obtenir pour chaque commune la station météo la plus proche en distance. Pour chaque commune on calcule la distance à chaque station et puis on séléctionne celle qui a la distance la plus faible.

In [5]:
plus_proche_station = np.zeros(len(df_communes))
for i in df_communes.index : 
    lat1 = df_communes.lat[i]
    lon1 = df_communes.lon[i]
    d_min = 100
    index_min = 0
    for j in df_stations_SE.index : 
        lat2 = df_stations_SE.lat[j]
        lon2 = df_stations_SE.lon[j]
        d = distance(lat1,lon1,lat2,lon2) 
        if d <= d_min :
            d_min = d
            index_min = j

    plus_proche_station[i] = df_stations_SE.number_sta[index_min]



- On va ensuite commencer la création du dataframe final ` df_final` en associant à chaque commune sa météo jounalière :

In [6]:
df_final = df_communes

df_final['number_sta'] = pd.DataFrame(plus_proche_station.astype(int))

df_meteo_moy = df_meteo_moy.rename(columns={'lat' : 'lat_sta', 'lon' : 'lon_sta'})

df_final = df_final.merge(df_meteo_moy, on = 'number_sta', how = 'left')

df_final['date'] = pd.to_datetime(df_final['date'])

display(df_final.head(10))


,index,code_commune_INSEE,lat,lon,region,number_sta,date,lat_sta,lon_sta,height_sta,dd,ff,precip,hu,td,t,psl
0,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-05,46.038,5.044,280.0,NaN,NaN,0.000000,NaN,NaN,281.837879,NaN
1,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-14,46.038,5.044,280.0,NaN,NaN,0.000000,NaN,NaN,272.222642,NaN
2,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-15,46.038,5.044,280.0,NaN,NaN,0.006667,NaN,NaN,273.077083,NaN
3,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-16,46.038,5.044,280.0,NaN,NaN,0.008333,NaN,NaN,277.585000,NaN
4,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-17,46.038,5.044,280.0,NaN,NaN,0.011667,NaN,NaN,277.890417,NaN
5,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-18,46.038,5.044,280.0,NaN,NaN,0.000833,NaN,NaN,278.067917,NaN
6,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-19,46.038,5.044,280.0,NaN,NaN,0.052500,NaN,NaN,280.643333,NaN
7,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-20,46.038,5.044,280.0,NaN,NaN,0.000000,NaN,NaN,279.035000,NaN
8,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-21,46.038,5.044,280.0,NaN,NaN,0.025833,NaN,NaN,281.076667,NaN
9,0,01001,46.153426,4.926114,Auvergne-Rhône-Alpes,1235001,2018-12-22,46.038,5.044,280.0,NaN,NaN,0.000000,NaN,NaN,283.481667,NaN


Pour finaliser notre base de donnée en vue de lui appliquer des algorithmes de régression, il nous faut maintenant ajouter une colonne "incendie" qui prend la valeur 1 s'il y a eu un incendie déclaré ce jour dans la commune et 0 sinon : 

In [ ]:
# Ajustements sur le dataframe incendie en vue de la jointure avec la base finale
df_incendies['Date de première alerte'] = pd.to_datetime(df_incendies['Date de première alerte'])
df_incendies['date'] = df_incendies['Date de première alerte'].dt.date
df_incendies = df_incendies.rename(columns={'Code INSEE' : 'code_commune_INSEE'})
df_incendies_18 = df_incendies[df_incendies['Année'] == 2018]

In [ ]:
df_final['date'] = df_final['date'].dt.date

df_final = pd.merge(df_final, df_incendies_18, how='left', on=['date', 'code_commune_INSEE'])

colonnes_a_verifier = ["Origine de l'alerte",
       'Moyens de première intervention', 'Surface parcourue (m2)',
       'Surface forêt (m2)', 'Surface maquis garrigues (m2)',
       'Autres surfaces naturelles hors forêt (m2)', 'Surfaces agricoles (m2)',
       'Autres surfaces (m2)', 'Surface autres terres boisées (m2)',
       'Surfaces non boisées naturelles (m2)',
       'Surfaces non boisées artificialisées (m2)',
       'Surfaces non boisées (m2)', 'Précision des surfaces',
       "Surface de feu à l'arrivée des secours > 0,1 ha",
       'Voie carrossable la plus proche',
       'Activité ou habitation la plus proche', 'Type de peuplement',
       'Connaissance', "Source de l'enquête", 'Nature',
       "Intervention de l'équipe RCCI", 'Décès ou bâtiments touchés',
       'Nombre de décès', 'Nombre de bâtiments totalement détruits',
       'Nombre de bâtiments partiellement détruits', 'Hygrométrie (%)',
       'Vitesse moyenne du vent (Km/h)', 'Direction du vent',
       'Température (°C)', 'Précision de la donnée',
       "Présence d'un contour valide"]  
       
# Si sur une même ligne toutes les valeurs colonnes relatives à l'incendie sont "nan" c'est qu'il n'y a eu d'incendie

df_final['incendie'] = 1-df_final[colonnes_a_verifier].isna().all(axis=1).astype(int)

display(df_final.sample(10))

-On va effectuer quelques vérifications pour voir si l'on a pas loupé des incendies par exemple :

In [ ]:
diff = list(set(df_incendies_18['code_commune_INSEE'])-set(df_final[df_final['incendie']== 1]['code_commune_INSEE']))
print("codes commune INSEE de communes ayant eu des incendies n'apparaissant pas dans la table finale :\n",diff)
print(f"Ils sont au nombre de {len(diff)}.")

Comme on peut le constater avec les deux premiers chiffres de ces codes communes INSEE qui correspondent au département, les départements où se situent ces communes  
ne sont pas des départements du Sud-Est et donc leurs feux de forêts n'entre pas dans le cadre de notre étude.  
Pour les  quelques exeptions qui restent, nous avons conclu après vérification dans la base de données de la météo qu'il s'agit en fait de ville où les données météo ne sont pas disponibles le jour de l'incendie. 